Problem Statement

In this assignment students need to predict whether a person makes over 50K per year
or not from classic adult dataset using XGBoost. The description of the dataset is as
follows:

Data Set Information:
Extraction was done by Barry Becker from the 1994 Census database. A set of
reasonably clean records was extracted using the following conditions: ((AAGE>16) &&
(AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

In [184]:
import numpy as np
import pandas as pd
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', skiprows = 1, header = None)
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
'occupation','relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week',
'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

In [185]:
# concatinate the training and testing data sets that were provided seperately
df = pd.concat([train_set, test_set], axis=0)
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [186]:
# Set values of > 50K to 1 and lessthan 50k to 0
df['wage_class'] = df['wage_class'].apply(lambda x: 1 if x==' >50K' else 0)

In [187]:
# Clean data by replacing ? with NaN
df.replace(' ?', np.nan, inplace=True)

In [188]:
df.isna().values.sum()

6465

In [189]:
df.fillna(0, inplace=True)

In [190]:
df.isna().values.sum()

0

In [191]:
# Drop education as education_number colume already represents similar information numeraically
df.drop('education', axis=1,inplace=True)

In [192]:
# Select the colums (categorical data) that need to be dummified
nominal_columns = ['workclass','marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
dummy_data = pd.get_dummies(df[nominal_columns])

In [193]:
# Merge back the dummfied colums to remaing ones
tmp_set = df.drop(nominal_columns, axis=1)
X = pd.concat([tmp_set, dummy_data], axis=1)
y = X['wage_class']
X.drop('wage_class', axis=1, inplace=True)


In [194]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

In [195]:
import numpy as np
from sklearn.datasets import load_svmlight_files
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier

In [196]:
# Params used for XGB
params = {
    'objective': 'binary:logistic',
    'max_depth': 2,
    'learning_rate': 1.0,
    'silent': 1.0,
    'n_estimators': 5
}


In [197]:
bst = XGBClassifier(**params).fit(X_train, y_train)

In [200]:
preds = bst.predict(X_test)
preds

/Users/swetha/anaconda2/envs/python35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([1, 0, 0, ..., 0, 0, 0])

In [209]:
from sklearn.metrics import accuracy_score
print("testing set auc:",accuracy_score(y_test, preds))

testing set auc: 0.8595557375371071
